In [1]:
import pandas as pd
import numpy as np

In [2]:
installs = pd.read_csv('../datos/installs.csv', dtype ={'device_language' : 'str', 'click_hash': object, 'wifi': object, 'trans_id': object})

In [3]:
# No tiene sentido quedarse con el pais, porque es uno solo
#No tiene sentido quedarse con la columna trans_id. El 98% de los valores son Null
#No tiene sentido quedarse con click_hash, el 99% de los valors son null
#No tiene sentido quedarse con event_uuid, el 78% de los valores son null
#No tiene sentido quedarse con kind, el 78% de los valores son null
#No tiene sentido quedarse con wifi, el 48% de los valores son null
#No tiene sentido quedarse con device_brand. El 42% son null
#No tiene sentido quedarse con la columna user_agent El 31% de los valores son Null
#¿Tiene sentido quedarse con la columna session_user_agent? El 3% de los valores son Null
# session_user_agent tiene el 3% de los valores null. Borro esas filas
del installs['trans_id']
del installs['device_countrycode']
del installs['click_hash']
del installs['event_uuid']
del installs['kind']
del installs['wifi']
del installs['device_brand']
del installs['user_agent']
del installs['session_user_agent']

##Borro las filas que tienen null en device_model y device_language. Juntas son el 5% de las filas en null
installs = installs[pd.notnull(installs['device_model'])]
installs = installs[pd.notnull(installs['device_language'])]

In [4]:
installs['created'] = pd.to_datetime(installs['created'], errors = 'coerce')
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [5]:
events = pd.read_csv('../datos/events.csv',
                     dtype={
                         'device_countrycode': 'category', 
                         'device_city': 'category',
                         'ref_type': 'int64',
                         'application_id': 'int64',
                         'device_os_version': 'category',
                         'device_brand': 'category',
                         'device_model': 'float64',
                         'session_user_agent': 'category',
                         'trans_id': 'category',
                         'user_agent': 'category',
                         'carrier': 'category',
                         'device_os': 'category',
                         'device_os_version': 'category',
                         'device_language': 'category',
                         'connection_type': 'category',
                         'wifi': 'bool'
                     })

In [6]:
del events['attributed']
del events['device_countrycode'] #Borrada porque hay un solo país
del events['trans_id'] #Borrada por tener muy pocos valores (37000/7000000)
del events['event_uuid'] #Borrada por tener todos los valores distintos
del events['device_os_version'] #Borrado por tener 70% de los valores nulos.
del events['device_brand'] #Borrado por tener 67% de los valores nulos
del events['device_city'] #Borrado por tener 76% de los valores nulos
del events['user_agent'] #Borrado por tener 57% de los valores nulos
del events['carrier'] #Borrado por tener 75% de los valores nulos
del events['device_os'] #Borrado por tener 76% de los valores nulos
del events['connection_type'] #Borrado por tener 77% de los valores nulos
del events['session_user_agent']
#------------------------------------------------------------------
events = events[pd.notnull(events['kind'])] #Borrados las filas nulas (0,5%)
#------------------------------------------------------------------
events = events[pd.notnull(events['device_model'])]

In [7]:
events['ref_hash'].value_counts().count()

348229

In [8]:
installs['ref_hash'].value_counts().count()

373799

In [55]:
installs.head()

,created,ref_type,ref_hash,total_apps,%implicit,%attributed,most_freq_lang,most_freq_app,model,ip,events_%wifi,event_apps,distinct_events,total_events
46742,2019-04-21 19:17:47.657,1891515180541284343,40621409780134,1,0.0,0.0,3.301378e+18,77,2.019322e+18,990380024993756535,0.0,9.0,3.0,9.0
398921,2019-04-18 21:11:50.326,1891515180541284343,41863526108385,4,0.5,0.0,6.977049e+18,65,1.658417e+18,7530145772806895848,0.0,32.0,7.0,32.0
249204,2019-04-18 21:11:51.966,1891515180541284343,41863526108385,4,0.5,0.0,6.977049e+18,65,1.658417e+18,7530145772806895848,0.0,32.0,7.0,32.0
112218,2019-04-18 21:17:11.946,1891515180541284343,41863526108385,4,0.5,0.0,6.977049e+18,65,1.658417e+18,7530145772806895848,0.0,32.0,7.0,32.0
112200,2019-04-18 21:17:16.531,1891515180541284343,41863526108385,4,0.5,0.0,6.977049e+18,65,1.658417e+18,7530145772806895848,0.0,32.0,7.0,32.0


In [10]:
eventos = events.groupby('ref_hash').agg({'wifi' : 'mean', 'application_id' : 'count', 'kind' : 'nunique', 'event_id' : 'count'}).reset_index()

In [12]:
events['mode_model'] = events.groupby('ref_hash')['device_model'].agg(lambda x: x.value_counts().index[0])
events['mode_lang'] = events.groupby('ref_hash')['device_language'].agg(lambda x: x.value_counts().index[0])

In [14]:
installs['total_apps'] = installs.groupby('ref_hash')['application_id'].transform('count')
installs['%implicit'] = installs.groupby('ref_hash')['implicit'].transform('mean')
installs['%attributed'] = installs.groupby('ref_hash')['attributed'].transform('mean')
installs['most_freq_lang'] = installs.groupby('ref_hash')['device_language'].transform(lambda x: x.mode().iloc[0])
installs['most_freq_app'] = installs.groupby('ref_hash')['application_id'].transform(lambda x: x.mode().iloc[0])
installs['model'] = installs.groupby('ref_hash')['device_model'].transform(lambda x: x.mode().iloc[0])
installs['ip'] = installs.groupby('ref_hash')['ip_address'].transform(lambda x: x.mode().iloc[0])

In [15]:
installs['events_%wifi'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['wifi'])
installs['event_apps'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['application_id'])
installs['distinct_events'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['kind'])
installs['total_events'] = installs['ref_hash'].map(eventos.set_index('ref_hash')['event_id'])

In [16]:
del installs['attributed']
del installs['implicit']
del installs['application_id']
del installs['device_model']
del installs['ip_address']
del installs['device_language']

In [18]:
installs.fillna(0, inplace=True)

In [19]:
installs = installs.sort_values(by = ['ref_hash', 'created'])

In [20]:
installs['most_freq_lang'] = installs['most_freq_lang'].astype('float64')

In [21]:
installs1 = installs[(installs['created'].dt.day < 21)]
installs2 = installs[(installs['created'].dt.day > 18) & (installs['created'].dt.day < 22)]
installs3 = installs[(installs['created'].dt.day > 19) & (installs['created'].dt.day < 23)]
installs4 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 24)]
installs5 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 25)]
installsCheck1 = installs[(installs['created'].dt.day > 20) & (installs['created'].dt.day < 23)]
installsCheck2 = installs[(installs['created'].dt.day > 21) & (installs['created'].dt.day < 24)]
installsCheck3 = installs[(installs['created'].dt.day > 22) & (installs['created'].dt.day < 25)]
installsCheck4 = installs[(installs['created'].dt.day > 23) & (installs['created'].dt.day < 26)]
installsCheck5 = installs[(installs['created'].dt.day > 24) & (installs['created'].dt.day < 27)]

In [22]:
installs1['sc'] = (installs1['created'] - pd.to_datetime('2019-04-18'))/np.timedelta64(1,'s')
installs2['sc'] = (installs2['created'] - pd.to_datetime('2019-04-19'))/np.timedelta64(1,'s')
installs3['sc'] = (installs3['created'] - pd.to_datetime('2019-04-20'))/np.timedelta64(1,'s')
installs4['sc'] = (installs4['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installs5['sc'] = (installs5['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck1['sc'] = (installsCheck1['created'] - pd.to_datetime('2019-04-21'))/np.timedelta64(1,'s')
installsCheck2['sc'] = (installsCheck2['created'] - pd.to_datetime('2019-04-22'))/np.timedelta64(1,'s')
installsCheck3['sc'] = (installsCheck3['created'] - pd.to_datetime('2019-04-23'))/np.timedelta64(1,'s')
installsCheck4['sc'] = (installsCheck4['created'] - pd.to_datetime('2019-04-24'))/np.timedelta64(1,'s')
installsCheck5['sc'] = (installsCheck5['created'] - pd.to_datetime('2019-04-25'))/np.timedelta64(1,'s')

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [23]:
installs1 = installs1.drop_duplicates('ref_hash', keep = 'first')
installs2 = installs2.drop_duplicates('ref_hash', keep = 'first')
installs3 = installs3.drop_duplicates('ref_hash', keep = 'first')
installs4 = installs4.drop_duplicates('ref_hash', keep = 'first')
installs5 = installs5.drop_duplicates('ref_hash', keep = 'first')
installsCheck1 = installsCheck1.drop_duplicates('ref_hash', keep = 'first')
installsCheck2 = installsCheck2.drop_duplicates('ref_hash', keep = 'first')
installsCheck3 = installsCheck3.drop_duplicates('ref_hash', keep = 'first')
installsCheck4 = installsCheck4.drop_duplicates('ref_hash', keep = 'first')
installsCheck5 = installsCheck5.drop_duplicates('ref_hash', keep = 'first')

In [24]:
del installs1['created']
del installs2['created']
del installs3['created']
del installs4['created']
del installs5['created']
del installsCheck1['created']
del installsCheck2['created']
del installsCheck3['created']
del installsCheck4['created']
del installsCheck5['created']

# Algoritmos

### XGBoost

In [25]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

#### Ventana 1

In [94]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha =10, n_estimators = 11)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52221.659961


#### Ventana 2

In [95]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52146.316026


#### Ventana 3

In [96]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 52157.097772


#### Ventana 4

In [97]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 53224.249049


#### Ventana 5

In [98]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
xg_reg.fit(X, y)
preds = xg_reg.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/fede/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 51092.387332


#### Target

In [31]:
target = pd.read_csv('../datos/target_competencia_ids.csv')

In [32]:
targets_sc = target[target['ref_hash'].str.contains('_sc')]
targets_st = target[target['ref_hash'].str.contains('_st')]

In [33]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].map(lambda x: str(x)[:-3])
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(np.int64)

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
installs_sin_dup = installs.drop_duplicates('ref_hash')

In [35]:
targets_sc_with_installs = targets_sc.merge(installs_sin_dup, how = 'left', on = 'ref_hash')

In [36]:
events.drop_duplicates('ref_hash', inplace= True)

In [45]:
nulls = targets_sc_with_installs[targets_sc_with_installs['created'].isnull()]
nulls['ref_type'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ref_type'])
nulls['model'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_model'])
nulls['most_freq_lang'] = nulls['ref_hash'].map(events.set_index('ref_hash')['device_language'])
nulls['most_freq_app'] = nulls['ref_hash'].map(events.set_index('ref_hash')['application_id'])
nulls['ip'] = nulls['ref_hash'].map(events.set_index('ref_hash')['ip_address'])
nulls['events_%wifi'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['wifi'])
nulls['event_apps'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['application_id'])
nulls['distinct_events'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['kind'])
nulls['total_events'] = nulls['ref_hash'].map(eventos.set_index('ref_hash')['event_id'])

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [48]:
targets_sc_with_installs.update(nulls)
targets_sc_with_installs.head()

,ref_hash,obj,created,ref_type,total_apps,%implicit,%attributed,most_freq_lang,most_freq_app,model,ip,events_%wifi,event_apps,distinct_events,total_events
0,1.000169e+18,0.0,2019-04-21 08:14:06.178,1.891515e+18,1.0,0.0,0.0,3.30138e+18,122.0,1.805456e+18,5.483720e+18,0.000000,32.0,4.0,32.0
1,1.000396e+18,0.0,2019-04-20 05:59:26.866,1.891515e+18,2.0,0.5,0.0,6.97705e+18,121.0,8.058599e+17,6.126933e+18,1.000000,2.0,2.0,2.0
2,1.003027e+18,0.0,2019-04-23 20:37:24.299,1.891515e+18,2.0,0.5,0.0,3.30138e+18,14.0,8.590181e+18,2.036110e+18,0.833333,54.0,10.0,54.0
3,1.006670e+18,0.0,2019-04-21 01:20:43.968,1.494519e+18,2.0,0.5,0.0,8.44142e+18,68.0,6.794880e+18,2.799308e+18,0.333333,3.0,2.0,3.0
4,1.007573e+18,0.0,2019-04-18 07:32:44.992,1.891515e+18,1.0,0.0,0.0,3.30138e+18,94.0,1.327864e+18,8.238829e+18,0.000000,6.0,1.0,6.0


In [116]:
targets_sc_with_installs.fillna({'total_apps' : 0, 'ref_type' : 0, '%attributed' : 0, '%implicit': 0, 'model' : 0,
                                 'most_freq_lang' : 0, 'ip' : 0, 'events_%wifi' : 0, 'event_apps' : 0, 
                                 'distinct_events' : 0, 'total_events' : 0, 'most_freq_app' : 0}, inplace = True)

In [50]:
del targets_sc_with_installs['created']
del targets_sc_with_installs['obj']

In [142]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_xgb = xg_reg.predict(X)

In [63]:
sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
sc_pred.index = np.arange(1, 8074, 2)
targets_st['obj'] = sc_pred[0]
target.update(targets_st)
target

/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ref_hash,obj
0,1000169251625791246_sc,80867.218750
1,1000169251625791246_st,79631.730000
2,1000395625957344683_sc,85391.179688
3,1000395625957344683_st,79679.830000
4,1003027494996471685_sc,81212.171875
5,1003027494996471685_st,82655.695000
6,1006670001679961544_sc,83880.101562
7,1006670001679961544_st,79636.414000
8,1007573308966476713_sc,82855.875000
9,1007573308966476713_st,79631.730000


In [64]:
target.to_csv('../datos/preds-xgboost-rf-knn-nuevos-features.csv', index = False)

### Bagging

In [85]:
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [141]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
bagging.fit(X, y)
preds = bagging.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67372.750068


In [143]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
bagging.fit(X, y)
preds = bagging.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 69828.794754


In [144]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
bagging.fit(X, y)
preds = bagging.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 70859.827872


In [145]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
bagging.fit(X, y)
preds = bagging.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68939.568416


In [147]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
bagging = BaggingRegressor(KNeighborsRegressor(), max_samples=0.5, max_features=0.5)
bagging.fit(X, y)
preds = bagging.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66511.270489


In [148]:
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_bagging = bagging.predict(X)

### AdaBoost

In [149]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
regr = AdaBoostRegressor(random_state=0, n_estimators=50, learning_rate=0.1)
regr.fit(X, y) 
preds = regr.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66205.001008


In [150]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67957.789315


In [151]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68742.307108


In [152]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67266.935307


In [153]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
regr.fit(X, y) 
preds = regr.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 64948.432210


In [154]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_adaboost = regr.predict(X)

### LGBM

In [120]:
import lightgbm as lgb

In [123]:
lgb_train = lgb.Dataset(installs1.iloc[:,:-1], installs1.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck1.iloc[:,:-1], installsCheck1.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck1.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l2: 4.32643e+09	valid_0's l1: 55436.3
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 4.32734e+09	valid_0's l1: 55438.4
[3]	valid_0's l2: 4.32699e+09	valid_0's l1: 55433.8
[4]	valid_0's l2: 4.32795e+09	valid_0's l1: 55435.3
[5]	valid_0's l2: 4.3291e+09	valid_0's l1: 55437
[6]	valid_0's l2: 4.33147e+09	valid_0's l1: 55446.4
[7]	valid_0's l2: 4.33372e+09	valid_0's l1: 55453.3
[8]	valid_0's l2: 4.33608e+09	valid_0's l1: 55462.3
[9]	valid_0's l2: 4.33856e+09	valid_0's l1: 55473
[10]	valid_0's l2: 4.34101e+09	valid_0's l1: 55483.8
[11]	valid_0's l2: 4.3434e+09	valid_0's l1: 55492.2
Early stopping, best iteration is:
[1]	valid_0's l2: 4.32643e+09	valid_0's l1: 55436.3
RMSE: 65775.576436


In [127]:
lgb_train = lgb.Dataset(installs2.iloc[:,:-1], installs2.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck2.iloc[:,:-1], installsCheck2.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck2.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l2: 4.60391e+09	valid_0's l1: 56837.9
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 4.60514e+09	valid_0's l1: 56843.4
[3]	valid_0's l2: 4.60507e+09	valid_0's l1: 56841.2
[4]	valid_0's l2: 4.60493e+09	valid_0's l1: 56835.3
[5]	valid_0's l2: 4.60613e+09	valid_0's l1: 56838.4
[6]	valid_0's l2: 4.60709e+09	valid_0's l1: 56841.7
[7]	valid_0's l2: 4.60841e+09	valid_0's l1: 56847.4
[8]	valid_0's l2: 4.6088e+09	valid_0's l1: 56846.6
[9]	valid_0's l2: 4.61026e+09	valid_0's l1: 56852.1
[10]	valid_0's l2: 4.61221e+09	valid_0's l1: 56860.1
[11]	valid_0's l2: 4.61175e+09	valid_0's l1: 56855.8
Early stopping, best iteration is:
[1]	valid_0's l2: 4.60391e+09	valid_0's l1: 56837.9
RMSE: 67852.095327


In [128]:
lgb_train = lgb.Dataset(installs3.iloc[:,:-1], installs3.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck3.iloc[:,:-1], installsCheck3.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck3.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l2: 4.7152e+09	valid_0's l1: 57631.5
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 4.71753e+09	valid_0's l1: 57639.1
[3]	valid_0's l2: 4.71796e+09	valid_0's l1: 57639.4
[4]	valid_0's l2: 4.71891e+09	valid_0's l1: 57639.1
[5]	valid_0's l2: 4.72151e+09	valid_0's l1: 57647.1
[6]	valid_0's l2: 4.72429e+09	valid_0's l1: 57656.3
[7]	valid_0's l2: 4.72704e+09	valid_0's l1: 57665.1
[8]	valid_0's l2: 4.72857e+09	valid_0's l1: 57667.8
[9]	valid_0's l2: 4.73201e+09	valid_0's l1: 57680.3
[10]	valid_0's l2: 4.7353e+09	valid_0's l1: 57691.3
[11]	valid_0's l2: 4.73752e+09	valid_0's l1: 57698.8
Early stopping, best iteration is:
[1]	valid_0's l2: 4.7152e+09	valid_0's l1: 57631.5
RMSE: 68667.326555


In [129]:
lgb_train = lgb.Dataset(installs4.iloc[:,:-1], installs4.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck4.iloc[:,:-1], installsCheck4.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck4.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l2: 4.53583e+09	valid_0's l1: 56382.1
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 4.53795e+09	valid_0's l1: 56390.9
[3]	valid_0's l2: 4.53819e+09	valid_0's l1: 56393.7
[4]	valid_0's l2: 4.53998e+09	valid_0's l1: 56402.9
[5]	valid_0's l2: 4.54169e+09	valid_0's l1: 56410.6
[6]	valid_0's l2: 4.54482e+09	valid_0's l1: 56427.7
[7]	valid_0's l2: 4.54776e+09	valid_0's l1: 56443.3
[8]	valid_0's l2: 4.55008e+09	valid_0's l1: 56455.5
[9]	valid_0's l2: 4.55204e+09	valid_0's l1: 56462.7
[10]	valid_0's l2: 4.55389e+09	valid_0's l1: 56472.5
[11]	valid_0's l2: 4.55734e+09	valid_0's l1: 56487.6
Early stopping, best iteration is:
[1]	valid_0's l2: 4.53583e+09	valid_0's l1: 56382.1
RMSE: 67348.581056


In [130]:
lgb_train = lgb.Dataset(installs5.iloc[:,:-1], installs5.iloc[:,-1])
lgb_eval = lgb.Dataset(installsCheck5.iloc[:,:-1], installsCheck5.iloc[:,-1], reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval, early_stopping_rounds=10)
preds = gbm.predict(installsCheck5.iloc[:,:-1], num_iteration=gbm.best_iteration)
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l2: 4.16867e+09	valid_0's l1: 54171.5
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 4.17498e+09	valid_0's l1: 54199.4
[3]	valid_0's l2: 4.17481e+09	valid_0's l1: 54193.9
[4]	valid_0's l2: 4.18067e+09	valid_0's l1: 54221.2
[5]	valid_0's l2: 4.18187e+09	valid_0's l1: 54219.4
[6]	valid_0's l2: 4.18366e+09	valid_0's l1: 54221.2
[7]	valid_0's l2: 4.18519e+09	valid_0's l1: 54221.1
[8]	valid_0's l2: 4.18683e+09	valid_0's l1: 54221.9
[9]	valid_0's l2: 4.1879e+09	valid_0's l1: 54219
[10]	valid_0's l2: 4.19224e+09	valid_0's l1: 54237.4
[11]	valid_0's l2: 4.1977e+09	valid_0's l1: 54259.8
Early stopping, best iteration is:
[1]	valid_0's l2: 4.16867e+09	valid_0's l1: 54171.5
RMSE: 64565.240891


In [155]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_lgbm = gbm.predict(X)

### Random Forests

In [157]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, criterion='mse', max_depth=5, min_samples_split=2, 
                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                           max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                           bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                           warm_start=False)

In [158]:
X, y = installs1.iloc[:,:-1], installs1.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck1.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck1.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 66190.135760


In [159]:
X, y = installs2.iloc[:,:-1], installs2.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck2.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck2.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67879.801619


In [160]:
X, y = installs3.iloc[:,:-1], installs3.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck3.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck3.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 68852.824544


In [161]:
X, y = installs4.iloc[:,:-1], installs4.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck4.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck4.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 67593.833753


In [162]:
X, y = installs5.iloc[:,:-1], installs5.iloc[:,-1]
rf.fit(X, y)
preds = rf.predict(installsCheck5.iloc[:,:-1])
rmse = np.sqrt(mean_squared_error(installsCheck5.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 64727.725242


In [163]:
targets_sc_with_installs['most_freq_lang'] = targets_sc_with_installs['most_freq_lang'].astype('float64')
X = targets_sc_with_installs[['ref_type', 'ref_hash', 'total_apps', '%implicit', '%attributed', 'most_freq_lang', 
                             'most_freq_app', 'model', 'ip', 'events_%wifi', 'event_apps', 'distinct_events', 
                             'total_events']]
predictions_sc_rf = rf.predict(X)

## Promedio las predicciones y actualizo el target

In [166]:
predictions_sc = np.add(np.add(np.add(np.add(predictions_sc_xgb,predictions_sc_bagging), predictions_sc_adaboost), predictions_sc_lgbm),predictions_sc_rf)
predictions_sc = predictions_sc / 5
targets_sc['obj'] = predictions_sc
predicciones_target_sc = target[target['ref_hash'].str.contains('_sc')]
predicciones_target_sc['obj'] = predictions_sc
target.update(predicciones_target_sc)
sc_pred = pd.read_csv('../datos/PrediccionesKNN-RF-XGBoost.txt', header = None)
sc_pred.index = np.arange(1, 8074, 2)
targets_st['obj'] = sc_pred[0]
target.update(targets_st)


/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/fede/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,ref_hash,obj
0,1000169251625791246_sc,119812.499918
1,1000169251625791246_st,79631.730000
2,1000395625957344683_sc,119704.066442
3,1000395625957344683_st,79679.830000
4,1003027494996471685_sc,122639.085441
5,1003027494996471685_st,82655.695000
6,1006670001679961544_sc,125853.029913
7,1006670001679961544_st,79636.414000
8,1007573308966476713_sc,113257.636739
9,1007573308966476713_st,79631.730000


In [167]:
target.to_csv('../datos/preds-promedios.csv', index = False)

In [168]:
target.count()

ref_hash    8074
obj         8074
dtype: int64